# Pre-computing various second-moment related quantities

This saves computation for M&M by precomputing and re-using quantitaties shared between iterations. It mostly saves $O(R^3)$ computations. This vignette shows results agree with the original version. Cannot use unit test due to numerical discrepency between `chol` of amardillo and R -- this has been shown problematic for some computations. I'll have to improve `mashr` for it.

In [ ]:
muffled_chol = function(x, ...)
withCallingHandlers(chol(x, ...),
                    warning = function(w) {
                      if (grepl("the matrix is either rank-deficient or indefinite", w$message))
                        invokeRestart("muffleWarning")
                    } )

In [2]:
set.seed(1)
library(mashr)
simdata = simple_sims(500,5,1)
data = mash_set_data(simdata$Bhat, simdata$Shat, alpha = 0)
U.c = cov_canonical(data)
grid = mashr:::autoselect_grid(data,sqrt(2))
Ulist = mashr:::normalize_Ulist(U.c)
xUlist = expand_cov(Ulist,grid,TRUE)
llik_mat0 = mashr:::calc_lik_rcpp(t(data$Bhat),t(data$Shat),data$V,
                             matrix(0,0,0), simplify2array(xUlist),F,T)$data
svs = data$Shat[1,] * t(data$V * data$Shat[1,])
sigma_rooti = list()
for (i in 1:length(xUlist)) sigma_rooti[[i]] = backsolve(muffled_chol(svs + xUlist[[i]], pivot=T), diag(nrow(svs)))
llik_mat = mashr:::calc_lik_common_rcpp(t(data$Bhat),
                                 simplify2array(sigma_rooti),
                                 F)$data

Loading required package: ashr


In [4]:
rows <- which(apply(llik_mat,2,function (x) any(is.infinite(x))))
if (length(rows) > 0)
    warning(paste("Some mixture components result in non-finite likelihoods,",
                          "either\n","due to numerical underflow/overflow,",
                          "or due to invalid covariance matrices",
                          paste(rows,collapse=", "), "\n"))
loglik_null = llik_mat[,1]
lfactors = apply(llik_mat,1,max)
llik_mat = llik_mat - lfactors
mixture_posterior_weights = mashr:::compute_posterior_weights(1/ncol(llik_mat), exp(llik_mat))
post0 = mashr:::calc_post_rcpp(t(data$Bhat), t(data$Shat), matrix(0,0,0), matrix(0,0,0), 
                              data$V,
                              matrix(0,0,0), matrix(0,0,0), 
                              simplify2array(xUlist),
                              t(mixture_posterior_weights),
                              T, 4)

In [5]:
Vinv = solve(svs)
U0 = list()
for (i in 1:length(xUlist)) U0[[i]] = xUlist[[i]] %*% solve(Vinv %*% xUlist[[i]] + diag(nrow(xUlist[[i]])))

In [8]:
post = mashr:::calc_post_precision_rcpp(t(data$Bhat), t(data$Shat), matrix(0,0,0), matrix(0,0,0), 
                              data$V,
                              matrix(0,0,0), matrix(0,0,0), 
                              Vinv,
                              simplify2array(U0),
                              t(mixture_posterior_weights),
                              4)

In [10]:
head(post$post_mean)

0.10648725,-0.27095076,-0.19287497,0.28341827,-0.102299771
0.02945312,-0.27311748,0.08146581,-0.38142642,-0.286370518
-0.13926952,0.37119813,-0.23913835,0.05792565,-0.011487166
-0.11395643,0.05432602,0.24782794,-0.03170211,0.081255488
0.06439769,-0.21287937,0.03375723,0.01161657,0.006370752
-0.12569260,0.34081892,-0.19934896,0.38732590,0.116490270


In [11]:
head(post0$post_mean)

0.10648725,-0.27095076,-0.19287497,0.28341827,-0.102299771
0.02945312,-0.27311748,0.08146581,-0.38142642,-0.286370518
-0.13926952,0.37119813,-0.23913835,0.05792565,-0.011487166
-0.11395643,0.05432602,0.24782794,-0.03170211,0.081255488
0.06439769,-0.21287937,0.03375723,0.01161657,0.006370752
-0.12569260,0.34081892,-0.19934896,0.38732590,0.116490270


In [12]:
head(post$post_cov)

[1]  0.287525181 -0.028692692 -0.012630509  0.085355181  0.006003125
[6] -0.028692692

In [13]:
head(post0$post_cov)

[1]  0.287525181 -0.028692692 -0.012630509  0.085355181  0.006003125
[6] -0.028692692